# layer.py对照
检查quantize和layers能否保证在各个模式下完全一致，包括structure_forward。
首先，利用暂存的Lenet权重，提取第一层并保存，并记录相关信息，同时随机生成一组input作为简单的测试

In [ ]:
import torch
weights = torch.load("../zoo/cifar10_lenet_params.pth")
layer1 = dict()
for name, params in weights.items():
    if name.startswith("layer_list.0"):
        layer1[name[len("layer_list.0."):]] = params
# save layer1
torch.save(layer1, "../zoo/cifar10_lenet_layer1.pth")
# save input
input = torch.clamp(torch.randn(1, 3, 6, 6), 0, 1)
torch.save(input, "../zoo/cifar10_lenet_input.pth")

In [ ]:
import torch
import MNSIM.Interface.quantize as quantize
from MNSIM.Interface.quantize import QuantizeLayer
hardware_config = {
    "xbar_size": 512,
    "input_bit": 2,
    "weight_bit": 1,
    "quantize_bit": 10,
}
layer_config = {
    "type": "conv",
    "in_channels": 3,
    "out_channels": 6,
    "kernel_size": 5,
}
quantize_config = {
    "weight_bit": 9,
    "activation_bit": 9,
    "point_shift": -2,
}
layer = QuantizeLayer(hardware_config, layer_config, quantize_config)
layer.load_state_dict(torch.load("../zoo/cifar10_lenet_layer1.pth"))
input = torch.load("../zoo/cifar10_lenet_input.pth")
# forward
quantize.last_activation_scale = 1 / 255.
quantize.last_activation_bit = 9
layer.eval()
output = layer.forward(input, method="SINGLE_FIX_TEST")
print(output)
torch.save(output, "quantize.pth")

In [ ]:
import torch
from MNSIM.Interface.layer import BaseWeightLayer
layer_ini = {
    "layer": {
        "in_channels": 3,
        "out_channels": 6,
        "kernel_size": 5,
    },
    "quantize": {
        "input": 9,
        "weight": 9,
        "output": 9,
    },
    "hardware": {
        "xbar_row": 512,
        "cell_bit": 1,
        "dac_bit": 2,
        "adc_bit": 10,
        "point_shift": -2,
    }
}
layer = BaseWeightLayer.get_class_("conv")(layer_ini)
layer.load_state_dict(torch.load("../zoo/cifar10_lenet_layer1.pth"), strict=False)
input = torch.load("../zoo/cifar10_lenet_input.pth")
# forward
layer.eval()
input_config = [torch.FloatTensor([9, 1/255.])]
output = layer.forward(input, method="SINGLE_FIX_TEST", input_config=input_config)
print(output)
torch.save(output, "layer.pth")


In [ ]:
import torch
a = torch.load("layer.pth")
b = torch.load("quantize.pth")
print(torch.max(a-b))